In [56]:
import re

def replace_first_instance(string, forward=True):
    """Replace first instance of a spelled-out digit."""
    digits = {'one': '1', 'two': '2', 'three': '3', 'four': '4', 'five': '5', 'six': '6', 'seven': '7', 'eight': '8', 'nine': '9'}
    for i in range(len(string)):
        # Return unchanged string if numeric digit appears first.
        if string[i].isdigit():
            return string
        for digit in digits:
            # Forward search and replace
            if forward and digit in string[:i+1]:
                return string.replace(digit, digits[digit], 1)
            # Backward search and replace
            if not forward and digit[::-1] in string[:i+1]:
                return string.replace(digit[::-1], digits[digit], 1)
    # Return unchanged string if no spelled-out digit appears.
    return string
    
def day_1(input_file):
    """Solve day 1 puzzle of Advent of Code 2023."""
    sums = [0, 0]
    with open(input_file, 'r') as file:
        for line in file:
            numbers = re.findall(r'\d', line)
            sums[0] += int(numbers[0] + numbers[-1])
            # Replace first instance of a spelled-out digit
            line = replace_first_instance(line)
            # Replace last instance of a spelled-out digit
            line = replace_first_instance(line[::-1], forward=False)[::-1]
            numbers = re.findall(r'\d', line)
            sums[1] += int(numbers[0] + numbers[-1])
    return sums

day_1('Day1.txt')  

[55477, 54431]

In [19]:
def day_2(input_file):
    """Solve day 2 puzzle of Advent of Code 2023."""
    part_1 = {'red': 12, 'green': 13, 'blue': 14}
    ID_sum = 0 
    powers = 0
    with open(input_file, 'r') as file:
        for i, line in enumerate(file):
            game = line.strip('\n').split(':', 1)[1].split(';')
            game_possible = True
            part_2 = {'red': 0, 'green': 0, 'blue': 0}
            for cubes in game:
                for cube in cubes.split(','):
                    number, color = cube.split(' ')[1:]
                    number = int(number)
                    if number > part_1[color]:
                        game_possible = False
                    part_2[color] = max(number, part_2[color])                       
            if game_possible:
                ID_sum += i+1
            powers += part_2['red'] * part_2['green'] * part_2['blue']
    return (ID_sum, powers)

day_2('Day2.txt')

(2528, 67363)

In [35]:
def day_3(input_file):
    """Solve day 3 puzzle of Advent of Code 2023."""
    nbors = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]
    parts = ()
    neighbours = ()
    symbols = {}
    with open(input_file, 'r') as file:
        for i, line in enumerate(file):
            part = ''
            neighbours_part = set()
            for j, char in enumerate(line):
                if char.isdigit():
                    part += char
                    neighbours_part.update([(i+nbor[0],j+nbor[1]) for nbor in nbors]) 
                else:
                    if part != '':
                        parts += (part,)
                        neighbours += (neighbours_part,)
                        part = ''
                        neighbours_part = set()                        
                    if char != '.' and char != '\n':
                        symbols[(i, j)] = char
                        
    sum = 0
    for i, part in enumerate(parts):
        for neighbour in neighbours[i]:
            if neighbour in symbols:
                sum += int(part)
                break
            
    gear_ratio = 0
    for coord in [key for key, value in symbols.items() if value == '*']:
        parts_gear = ()
        for i, neighbour in enumerate(neighbours):
            if coord in neighbour: 
                parts_gear += (int(parts[i]),)
        if len(parts_gear) == 2:
            gear_ratio += parts_gear[0] * parts_gear[1]
        
    return (sum, gear_ratio)                
    
day_3('Day3.txt')
    

(559667, 86841457)

In [57]:
def day_4(input_file):
    """Solve day 4 puzzle of Advent of Code 2023.""" 
    sum = 0
    num_cards = 0
    stash = []      
    with open(input_file, 'r') as file:
        for i, line in enumerate(file):
            winning_nums, nums = line.strip('\n').split(': ')[1].split(' | ')
            winning_nums = [int(num) for num in winning_nums.split()]
            nums = [int(num) for num in nums.split()]
            matching_nums = len([num for num in nums if num in winning_nums])
            sum += int(2**matching_nums-1)
            num_card = 1
            if len(stash) > 0:
                num_card += stash[0]
                stash.pop(0)
            num_cards += num_card
            for i in range(matching_nums):
                try:
                    stash[i] += num_card
                except IndexError:
                    stash.append(num_card)
    return((sum, num_cards))

day_4('Day4.txt')
    

(51131, 19499881)

In [44]:
def day_5(input_file):
    """Solve day 5 puzzle of Advent of Code 2023."""  
    with open(input_file, 'r') as file:
        for line in file:
            if line.startswith('seeds'):
                numbers_1 = [int(number) for number in line.strip('seeds: ').strip('\n').split()]
                numbers_2 = [(a, b) for a, b in zip(numbers_1[::2], numbers_1[1::2])]
            elif line[0].isalpha():
                try:
                    numbers_1 = new_numbers_1.copy()
                    numbers_2 = new_numbers_2.copy()
                except NameError:
                    pass
                new_numbers_1 = numbers_1.copy()
                new_numbers_2 = numbers_2.copy()
            elif line[0].isdigit():
                dest, source, length = [int(number) for number in line.strip('\n').split()]
                for i, number in enumerate(numbers_1):
                    if number >= source and number < source + length:
                        new_numbers_1[i] = dest - source + number
                for i, (number, range) in enumerate(numbers_2):
                    # If overlap but number < source, cut off part of
                    # (number, range) < source
                    if number < source and number + range > source:
                        numbers_2.append((number, source - number))
                        new_numbers_2.append((number, source - number))  
                        range -= source - number 
                        number = source
                    # If overlap but number + range > source + length, cut off part of
                    # (number, range) > source + length
                    if number < source + length and number + range > source + length:
                        numbers_2.append((source + length, number + range - source - length))
                        new_numbers_2.append((source + length, number + range - source - length))  
                        range -= number + range - source - length
                    if number >= source and number < source + length:
                        numbers_2[i] = (number, range)
                        new_numbers_2[i] = (dest - source + number, range)
                    
    return (min(new_numbers_1), min([a for (a, b) in new_numbers_2]))  
                 
day_5('Day5.txt')

(177942185, 69841803)

In [31]:
def day_6(input_file):
    """Solve day 6 puzzle of Advent of Code 2023."""
    product= 1
    with open(input_file, 'r') as file:
        for line in file:
            if line.startswith('Time:'):
                times = [int(time) for time in line.strip('\n').strip('Time:').split()]
                times.append(int(line.strip('\n').strip('Time:').replace(' ', '')))
            elif line.startswith('Distance:'):
                distances = [int(time) for time in line.strip('\n').strip('Distance:').split()]
                distances.append(int(line.strip('\n').strip('Distance:').replace(' ', '')))
    # time = time_move + time_charge
    # distance = time_move * speed = time_move * time_charge
    for time, distance in zip(times, distances):
        number = 0
        for time_charge in range(time):
            dist = time_charge * (time - time_charge)
            if dist > distance:
                number += 1
        if time == times[-1]:
            return (product, number)
        product *= number
        
day_6('Day6.txt')

(1083852, 23501589)